<a href="https://colab.research.google.com/github/Ads369/Ads_2s/blob/main/8_3_%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D1%8F%D1%8F_%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Навигация по уроку**

1. [Библиотеки визуализации данных в Python](https://colab.research.google.com/drive/1IbLhzlqiJhhPAErmdQ9wuIUdDaSRlNUF)
2. [Задача об акциях Tesla](https://colab.research.google.com/drive/1jukOs54u301WtyQS-SbRxWxDstTCjEBq)
3. Домашняя работа

Используя датасет о стоимости акций Сбербанка с 01.01.2013 года:

https://storage.yandexcloud.net/academy.ai/SBER.csv

визуализируйте индикатор "Полосы Боллинджера", проанализируйте график, и предложите вариант торговли акциями Сбербанка с помощью этого инструмента.

**Подсказка.**
Индикатор выглядит как полоса из трех линий:

* линия посередине — это простая скользящая средняя (SMA) с периодом `ma_size`, обычно около 20 дней;

* верхняя и нижняя линии (BB) — построены на основе SMА, но к нему добавлено стреднеквадратичное отклонение:

```
 SMA = data['close'].rolling(ma_size).mean()
 BB_UP = SMA + data['close'].rolling(ma_size).std() * bol_size
 BB_DOWN = SMA - data['close'].rolling(ma_size).std() * bol_size
   
```

где bol_size - ширина коридора, подбирается по графику. Выберите такое его значение, чтобы по графику можно было принимать торговые решения.

In [ ]:
!wget https://storage.yandexcloud.net/academy.ai/SBER.csv


In [5]:
import os
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots

bol_size = 2
ma_size = 20

In [1]:
df = pd.read_csv("SBER.csv", sep=";")

df["DATE"] = pd.to_datetime(df["DATE"], format="%Y%m%d")
df["SMA"] = df["CLOSE"].rolling(ma_size).mean()
df["BB_UP"] = df["SMA"] + df["CLOSE"].rolling(ma_size).std() * bol_size
df["BB_DOWN"] = df["SMA"] - df["CLOSE"].rolling(ma_size).std() * bol_size
df["BB_WIDTH"] = df["BB_UP"] - df["BB_DOWN"]


fig = go.Figure()
fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
fig.update_layout(
    title="Акции сбербанка",
    xaxis_title="Date",
    yaxis_title="Price",
    xaxis_rangeslider_visible=False,
)

# Рисуем свечки
fig.add_trace(
    go.Candlestick(
        x=df["DATE"],
        open=df["OPEN"],
        high=df["HIGH"],
        low=df["LOW"],
        close=df["CLOSE"],
        name="Price",
    ),
    row=1,
    col=1,
)

# Рисуем Полосы Боллинджера
for param in ["SMA", "BB_UP", "BB_DOWN"]:
    fig.add_trace(
        go.Scatter(
            x=df["DATE"],
            y=df[param],
            showlegend=False,
            line_color="gray",
            mode="lines",
            line={"dash": "dash"},
            marker_line_width=2,
            marker_size=10,
            opacity=0.8,
            name=param,
        ),
        row=1,
        col=1,
    )

# Рисуем Сжатие и Расширение полос 
fig.add_trace(
    go.Scatter(x=df["DATE"], y=df["BB_WIDTH"], mode="lines", name="BB Width"),
    row=2,
    col=1,
)

# Рисуем пересечение цены с верхней или нижней полосой
upper_cross = df[df["CLOSE"] > df["BB_UP"]]
lower_cross = df[df["CLOSE"] < df["BB_DOWN"]]

fig.add_trace(
    go.Scatter(
        x=upper_cross["DATE"],
        y=upper_cross["CLOSE"],
        mode="markers",
        name="Продавай",
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=lower_cross["DATE"],
        y=lower_cross["CLOSE"],
        mode="markers",
        name="Покупай",
    ),
    row=1,
    col=1,
)


# FIX RangeSlider
fig.update_layout(
    legend_orientation="h",
    xaxis2_rangeslider_visible=True,
    xaxis2_rangeslider_thickness=0.1,
)


fig.show()


NameError: name 'pd' is not defined

Сжатие полос - указывает на то что скоро, цены могут поменяться.

Расширение полос - указывает на то что тренд скорей всего сохранится.

Пересечение с нижней или верхней линией - указывает на то что цены развернутся.